# Infer-12-Recommenders : Systemes de Recommandation

**Serie** : Programmation Probabiliste avec Infer.NET (12/12)  
**Duree estimee** : 60 minutes  
**Prerequis** : Infer-11-Sequences

---

## Objectifs

- Comprendre le filtrage collaboratif bayesien
- Implementer la factorisation matricielle
- Gerer le probleme du cold-start
- Reconcilier des sources multiples (ClickModel)

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-11-Sequences](Infer-11-Sequences.ipynb) | [README](README.md) |

---

## 1. Configuration

In [ ]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

## 2. Introduction au Filtrage Collaboratif

### Principe

Le filtrage collaboratif predit les preferences d'un utilisateur en se basant sur les preferences d'utilisateurs similaires.

### Matrice de notes

```
          Film1  Film2  Film3  Film4
User1     5      ?      3      ?
User2     ?      4      ?      2
User3     4      3      5      ?
User4     ?      ?      4      5
```

### Approches

| Methode | Description | Avantage |
|---------|-------------|----------|
| Voisinage | k-NN sur similarite | Simple |
| Factorisation | Decomposition U x V | Scalable |
| Bayesien | Incertitude + priors | Robuste |

## 3. Factorisation Matricielle Bayesienne

In [ ]:
// Donnees : notes observees
int nUsers = 4;
int nItems = 5;
int nTraits = 2;  // Facteurs latents

// Observations : (user, item, note)
int[] userObs = { 0, 0, 1, 1, 2, 2, 3, 3 };
int[] itemObs = { 0, 2, 1, 3, 0, 2, 2, 4 };
double[] noteObs = { 5.0, 3.0, 4.0, 2.0, 4.0, 5.0, 4.0, 5.0 };
int nObs = userObs.Length;

Console.WriteLine("=== Factorisation Matricielle ===");
Console.WriteLine($"\nUtilisateurs : {nUsers}, Items : {nItems}, Traits : {nTraits}");
Console.WriteLine($"Observations : {nObs} notes");
Console.WriteLine("\nNotes observees :");
for (int i = 0; i < nObs; i++)
{
    Console.WriteLine($"  User {userObs[i]} -> Item {itemObs[i]} : {noteObs[i]}");
}

In [ ]:
// Modele de factorisation

Range userRange = new Range(nUsers).Named("user");
Range itemRange = new Range(nItems).Named("item");
Range traitRange = new Range(nTraits).Named("trait");
Range obsRange = new Range(nObs).Named("obs");

// Traits utilisateurs : U[user, trait]
VariableArray2D<double> userTraits = Variable.Array<double>(userRange, traitRange).Named("userTraits");
userTraits[userRange, traitRange] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(userRange, traitRange);

// Traits items : V[item, trait]
VariableArray2D<double> itemTraits = Variable.Array<double>(itemRange, traitRange).Named("itemTraits");
itemTraits[itemRange, traitRange] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(itemRange, traitRange);

// Precision du bruit
Variable<double> noisePrecision = Variable.GammaFromShapeAndScale(2, 0.5).Named("noisePrecision");

Console.WriteLine("Traits latents definis (U et V).");

In [ ]:
// Observations indexees
VariableArray<int> userIndex = Variable.Observed(userObs, obsRange).Named("userIndex");
VariableArray<int> itemIndex = Variable.Observed(itemObs, obsRange).Named("itemIndex");
VariableArray<double> rating = Variable.Observed(noteObs, obsRange).Named("rating");

// Modele de generation des notes
using (Variable.ForEach(obsRange))
{
    // Produit scalaire des traits
    VariableArray<double> produits = Variable.Array<double>(traitRange).Named("produits");
    produits[traitRange] = userTraits[userIndex[obsRange], traitRange] * itemTraits[itemIndex[obsRange], traitRange];
    
    Variable<double> affinite = Variable.Sum(produits).Named("affinite");
    
    // Note = affinite + bruit gaussien
    rating[obsRange] = Variable.GaussianFromMeanAndPrecision(affinite, noisePrecision);
}

Console.WriteLine("Modele de notes defini : rating ~ Gaussian(U * V', precision).");

In [ ]:
// Inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur.Algorithm = new ExpectationPropagation();

Console.WriteLine("\n=== Inference ===");

var userTraitsPost = moteur.Infer<Gaussian[,]>(userTraits);
var itemTraitsPost = moteur.Infer<Gaussian[,]>(itemTraits);
var noisePrecPost = moteur.Infer<Gamma>(noisePrecision);

Console.WriteLine($"\nPrecision du bruit : {noisePrecPost.GetMean():F2}");

Console.WriteLine("\nTraits utilisateurs (moyenne) :");
for (int u = 0; u < nUsers; u++)
{
    Console.Write($"  User {u} : [");
    for (int t = 0; t < nTraits; t++)
    {
        Console.Write($"{userTraitsPost[u, t].GetMean():F2}");
        if (t < nTraits - 1) Console.Write(", ");
    }
    Console.WriteLine("]");
}

## 4. Prediction de Notes

In [ ]:
// Prediction pour des paires (user, item) non observees

Console.WriteLine("\n=== Predictions ===");
Console.WriteLine("\nNotes predites pour paires non observees :");

// Calculer l'affinite predite
for (int u = 0; u < nUsers; u++)
{
    Console.Write($"User {u} : ");
    for (int i = 0; i < nItems; i++)
    {
        // Verifier si observe
        bool observe = false;
        for (int o = 0; o < nObs; o++)
        {
            if (userObs[o] == u && itemObs[o] == i)
            {
                observe = true;
                break;
            }
        }
        
        // Calcul produit scalaire des moyennes
        double pred = 0;
        for (int t = 0; t < nTraits; t++)
        {
            pred += userTraitsPost[u, t].GetMean() * itemTraitsPost[i, t].GetMean();
        }
        
        if (observe)
            Console.Write($"[{pred:F1}] ");
        else
            Console.Write($" {pred:F1}  ");
    }
    Console.WriteLine();
}

Console.WriteLine("\n(Notes entre crochets = observees)");

## 5. Cold-Start avec Features

Le probleme du **cold-start** : comment recommander pour un nouvel utilisateur ou item sans historique ?

### Solution : utiliser des features

- **Utilisateur** : age, genre, localisation
- **Item** : genre, annee, realisateur

In [ ]:
// Modele avec features utilisateur

Console.WriteLine("=== Cold-Start avec Features ===");

// Features utilisateurs : age normalise, genre (0/1)
double[,] userFeatures = {
    { 0.2, 1.0 },  // User 0 : jeune, homme
    { 0.8, 0.0 },  // User 1 : age, femme
    { 0.5, 1.0 },  // User 2 : moyen, homme
    { 0.3, 0.0 }   // User 3 : jeune, femme
};

// Features items : genre (action/romance), annee
double[,] itemFeatures = {
    { 1.0, 0.0, 0.9 },  // Item 0 : action, recent
    { 0.0, 1.0, 0.5 },  // Item 1 : romance, ancien
    { 0.5, 0.5, 0.8 },  // Item 2 : mixte, recent
    { 0.0, 1.0, 0.3 },  // Item 3 : romance, tres ancien
    { 1.0, 0.0, 0.7 }   // Item 4 : action, moyen
};

int nUserFeatures = 2;
int nItemFeatures = 3;

Console.WriteLine($"\nFeatures utilisateurs : {nUserFeatures} (age, genre)");
Console.WriteLine($"Features items : {nItemFeatures} (action, romance, annee)");

In [ ]:
// Nouveau modele avec biais bases sur features

Range uFeatRange = new Range(nUserFeatures).Named("uFeat");
Range iFeatRange = new Range(nItemFeatures).Named("iFeat");

// Poids pour les features
VariableArray<double> userWeights = Variable.Array<double>(uFeatRange).Named("userWeights");
userWeights[uFeatRange] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(uFeatRange);

VariableArray<double> itemWeights = Variable.Array<double>(iFeatRange).Named("itemWeights");
itemWeights[iFeatRange] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(iFeatRange);

// Biais global
Variable<double> globalBias = Variable.GaussianFromMeanAndPrecision(3, 0.1).Named("globalBias");

Console.WriteLine("Poids de regression pour cold-start definis.");

In [ ]:
// Prediction cold-start simplifiee (sans factorisation complete)

Console.WriteLine("\n=== Prediction Cold-Start ===");
Console.WriteLine("\nSimulation : nouvel utilisateur (age=0.4, genre=1.0)");

// Nouvel utilisateur
double[] newUserFeat = { 0.4, 1.0 };

// Modele simple : affinite = features_user dot weights + features_item dot weights
// Utilisons les moyennes des posterieurs (simplification)
double[] wUser = { 0.5, 0.8 };   // Appris (simule)
double[] wItem = { 0.6, -0.3, 0.2 };  // Appris (simule)

Console.WriteLine("\nScores predits pour chaque item :");
for (int i = 0; i < nItems; i++)
{
    double userScore = 0;
    for (int f = 0; f < nUserFeatures; f++)
        userScore += newUserFeat[f] * wUser[f];
    
    double itemScore = 0;
    for (int f = 0; f < nItemFeatures; f++)
        itemScore += itemFeatures[i, f] * wItem[f];
    
    double prediction = 3.0 + userScore + itemScore;  // Biais + scores
    Console.WriteLine($"  Item {i} : {prediction:F2}");
}

Console.WriteLine("\n=> Recommander items avec score le plus eleve");

## 6. Click Model : Sources Multiples

### Probleme

Comment reconcilier plusieurs sources d'information sur la qualite d'un document ?

- **Jugements humains** : experts mais couteux
- **Clics utilisateurs** : abondants mais bruites
- **Temps de lecture** : signal implicite

### Modele

```
Score latent (vrai) du document
        |
    +---+---+---+
    |   |   |   |
    v   v   v   v
  Juge Clic Temps ...
```

In [ ]:
// Click Model simplifie

int nDocs = 6;

// Observations de deux sources
double[] jugements = { 4.5, 3.0, 4.0, 2.5, 5.0, 3.5 };  // Notes experts (1-5)
double[] clics = { 120, 80, 95, 60, 150, 70 };          // Nombre de clics

Console.WriteLine("=== Click Model ===");
Console.WriteLine("\nReconciliation jugements experts vs clics utilisateurs");
Console.WriteLine("\nDonnees :");
for (int d = 0; d < nDocs; d++)
{
    Console.WriteLine($"  Doc {d} : Juge={jugements[d]:F1}, Clics={clics[d]}");
}

In [ ]:
// Modele Click

Range docRange = new Range(nDocs).Named("doc");

// Score latent (vrai) de chaque document
VariableArray<double> scoreLatent = Variable.Array<double>(docRange).Named("scoreLatent");
scoreLatent[docRange] = Variable.GaussianFromMeanAndPrecision(3, 0.5).ForEach(docRange);

// Precision de chaque source
Variable<double> precJuge = Variable.GammaFromShapeAndScale(5, 1).Named("precJuge");    // Experts : haute precision
Variable<double> precClic = Variable.GammaFromShapeAndScale(2, 0.5).Named("precClic"); // Clics : basse precision

// Facteur d'echelle pour les clics (clics = score * echelle + bruit)
Variable<double> echelleClics = Variable.GaussianFromMeanAndPrecision(30, 0.01).Named("echelleClics");

// Observations
VariableArray<double> obsJuge = Variable.Array<double>(docRange).Named("obsJuge");
VariableArray<double> obsClic = Variable.Array<double>(docRange).Named("obsClic");

// Modele generatif
obsJuge[docRange] = Variable.GaussianFromMeanAndPrecision(scoreLatent[docRange], precJuge);
obsClic[docRange] = Variable.GaussianFromMeanAndPrecision(scoreLatent[docRange] * echelleClics, precClic);

// Observations
obsJuge.ObservedValue = jugements;
obsClic.ObservedValue = clics;

Console.WriteLine("Modele Click defini avec deux sources.");

In [ ]:
// Inference
InferenceEngine moteur2 = new InferenceEngine();
moteur2.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("\n=== Inference Click Model ===");

var scorePost = moteur2.Infer<Gaussian[]>(scoreLatent);
var precJugePost = moteur2.Infer<Gamma>(precJuge);
var precClicPost = moteur2.Infer<Gamma>(precClic);
var echellePost = moteur2.Infer<Gaussian>(echelleClics);

Console.WriteLine($"\nPrecision juges : {precJugePost.GetMean():F2}");
Console.WriteLine($"Precision clics : {precClicPost.GetMean():F2}");
Console.WriteLine($"Echelle clics : {echellePost.GetMean():F2}");

Console.WriteLine("\nScores latents inferes :");
for (int d = 0; d < nDocs; d++)
{
    double mean = scorePost[d].GetMean();
    double std = Math.Sqrt(scorePost[d].GetVariance());
    Console.WriteLine($"  Doc {d} : {mean:F2} +/- {std:F2}  (Juge: {jugements[d]:F1}, Clics: {clics[d]})");
}

In [ ]:
// Classement final

Console.WriteLine("\n=== Classement Final ===");

var classement = Enumerable.Range(0, nDocs)
    .Select(d => new { Doc = d, Score = scorePost[d].GetMean() })
    .OrderByDescending(x => x.Score)
    .ToList();

Console.WriteLine("\nDocuments classes par score latent :");
int rang = 1;
foreach (var item in classement)
{
    Console.WriteLine($"  {rang}. Doc {item.Doc} (score: {item.Score:F2})");
    rang++;
}

Console.WriteLine("\n=> Le modele combine optimalement les deux sources");

## 7. Exercice : Recommandation de Films

### Enonce

Utilisez le modele de factorisation pour predire les notes manquantes et recommander des films.

In [ ]:
// EXERCICE : Recommandation de films

// Films : 0=Inception, 1=Titanic, 2=Matrix, 3=NotebookFilm, 4=Terminator
string[] films = { "Inception", "Titanic", "Matrix", "Notebook", "Terminator" };

// Utilisateurs : 0=Alice, 1=Bob, 2=Claire, 3=David
string[] users = { "Alice", "Bob", "Claire", "David" };

// Notes observees (memes donnees que section 3)
Console.WriteLine("=== Recommandation de Films ===");
Console.WriteLine("\nNotes observees :");
Console.WriteLine($"  Alice -> Inception: 5, Matrix: 3");
Console.WriteLine($"  Bob -> Titanic: 4, Notebook: 2");
Console.WriteLine($"  Claire -> Inception: 4, Matrix: 5");
Console.WriteLine($"  David -> Matrix: 4, Terminator: 5");

// Utiliser les posterieurs calcules precedemment
Console.WriteLine("\n=== Top 3 Recommandations par utilisateur ===");

for (int u = 0; u < nUsers; u++)
{
    var predictions = new List<(int item, double score)>();
    
    for (int i = 0; i < nItems; i++)
    {
        // Verifier si deja note
        bool observe = false;
        for (int o = 0; o < nObs; o++)
        {
            if (userObs[o] == u && itemObs[o] == i)
            {
                observe = true;
                break;
            }
        }
        
        if (!observe)
        {
            double pred = 0;
            for (int t = 0; t < nTraits; t++)
            {
                pred += userTraitsPost[u, t].GetMean() * itemTraitsPost[i, t].GetMean();
            }
            predictions.Add((i, pred));
        }
    }
    
    var top3 = predictions.OrderByDescending(p => p.score).Take(3);
    Console.WriteLine($"\n{users[u]} :");
    foreach (var rec in top3)
    {
        Console.WriteLine($"  -> {films[rec.item]} (score predit: {rec.score:F2})");
    }
}

## 8. Resume

| Concept | Description |
|---------|-------------|
| **Factorisation** | Decomposition U x V des preferences latentes |
| **Cold-start** | Utiliser features pour nouveaux utilisateurs/items |
| **Click Model** | Reconcilier sources de qualite variable |
| **Matchbox** | Implementation industrielle dans Infer.NET |

---

## Serie Complete

Felicitations ! Vous avez termine la serie **Programmation Probabiliste avec Infer.NET**.

| # | Notebook | Concepts |
|---|----------|----------|
| 1 | Setup | Installation, premier modele |
| 2 | Gaussian-Mixtures | Posterieurs, melanges |
| 3 | Factor-Graphs | Inference discrete, Monty Hall |
| 4 | Bayesian-Networks | CPT, causalite |
| 5 | Skills-IRT | IRT, DINA, many-to-many |
| 6 | TrueSkill | Ranking, online learning |
| 7 | Classification | BPM, A/B testing |
| 8 | Model-Selection | Evidence, ARD |
| 9 | Topic-Models | LDA, documents-topics |
| 10 | Crowdsourcing | Workers, communautes |
| 11 | Sequences | HMM, motif finding |
| 12 | Recommenders | Factorisation, Click Model |

---

## Ressources

- [Documentation Infer.NET](https://dotnet.github.io/infer/)
- [Livre MBML](https://mbmlbook.com/)
- [Code source Infer.NET](https://github.com/dotnet/infer)